In [1]:
import pandas as pd
import numpy as np

In [2]:
movies = pd.read_csv('../data/imdb-processed.csv')

In [12]:
movies['total_votes'] = movies.loc[:, 'rating_1':'rating_10'].sum(axis='columns')

We made a variable `total_votes` with the total number of user votes.

In [13]:
def calc_polarization(row):
    total_score = 0
    for rating_1 in range(1, 11):
        for rating_2 in range(rating_1 + 1, 11):
            total_score += row[f'rating_{rating_1}'] / row['total_votes'] * row[f'rating_{rating_2}'] / row['total_votes'] * (rating_2 - rating_1)

    return total_score

In [14]:
movies['polarization_score'] = movies.apply(calc_polarization, axis='columns')

We made a variable `polarization_score` with the polarization score calculated using Esteban and Ray's algorithm.

In [16]:
def calc_country_rating(country_idx, row):
    total_rating = 0
    total_votes = 0
    for rating in range(1, 11):
        total_rating += rating * row[f'country_{country_idx}_rating_{rating}']
        total_votes += row[f'country_{country_idx}_rating_{rating}']
    return total_rating / total_votes

In [17]:
for i in range(5):
    movies[f'country_{i}_rating'] = movies.apply(lambda row: calc_country_rating(i, row), axis='columns')

In [36]:
def calc_ptp(row):
    ptp = np.ptp(row.loc['country_0_rating':'country_4_rating'])

    max_country_idx = np.argmax(row.loc['country_0_rating':'country_4_rating'])
    min_country_idx = np.argmin(row.loc['country_0_rating':'country_4_rating'])
    country_ptp_pair = f"{row[f'country_{min_country_idx}']} < {row[f'country_{max_country_idx}']}"
    
    return pd.Series([ptp, country_ptp_pair])

In [37]:
movies[['country_ptp', 'country_ptp_pair']] = movies.apply(calc_ptp, axis='columns')

In [38]:
movies.sort_values('country_ptp', ascending=False).head(25)

,id,title,genres,weighted_rating,release_year,rating_1,rating_2,rating_3,rating_4,rating_5,...,country_4_rating_10,total_votes,polarization_score,country_0_rating,country_1_rating,country_2_rating,country_3_rating,country_4_rating,country_ptp,country_ptp_pair
3314,tt4776998,The Promise,"Action,Adventure,Drama",6.1,2017,71664,785,464,775,1792,...,4487,179364,2.203718,8.853424,1.372856,9.621135,7.973952,9.920069,8.547214,TR < AM
4299,tt11555492,Farha,Drama,8.0,2022,9534,177,97,127,268,...,4083,90080,0.981824,9.418896,2.721110,9.675454,9.617452,9.901685,7.180575,IL < JO
4326,tt7668842,Enes Batur: Imagination or Reality?,Comedy,2.0,2018,21542,748,526,347,471,...,176,56291,2.186839,5.369399,3.201413,7.219048,3.030189,9.915730,6.885542,GB < MY
4322,tt10554232,Dara of Jasenovac,"Drama,War",8.1,2021,3818,262,176,200,350,...,449,82083,0.560421,9.872591,9.197059,9.094617,9.785560,3.023002,6.849588,HR < RS
4314,tt27719898,Kaiva,"Action,Drama,Romance",8.5,2023,7908,1162,16,28,37,...,1,68958,1.159006,8.632270,5.288793,7.434343,2.772727,2.846154,5.859543,CA < IN
4271,tt15979666,The Archies,"Comedy,Drama,Musical",4.2,2023,7096,918,579,601,754,...,36,53125,1.472463,8.523331,3.764650,3.804031,4.066975,3.327014,5.196317,CA < IN
2683,tt0077928,Midnight Express,"Biography,Crime,Drama",7.5,1978,5628,684,675,1175,2725,...,280,90637,1.065465,7.436350,7.554844,3.224610,8.212210,7.648483,4.987600,TR < GR
4272,tt9637132,Samrat Prithviraj,"Action,Drama,History",5.6,2022,7574,964,586,637,927,...,53,53725,1.497891,8.384095,5.233802,5.597938,3.492119,4.573034,4.891976,BD < IN
7,tt20221436,Emilia Pérez,"Comedy,Crime,Drama",5.5,2024,24930,3995,3366,3753,5153,...,197,73792,1.737361,4.775475,2.465527,1.949050,5.211796,6.801721,4.852670,MX < FR
4328,tt10280276,Coolie No. 1,"Action,Comedy,Romance",4.2,2020,46055,1455,760,576,525,...,31,74467,1.910510,3.133739,6.429201,1.851842,4.394161,1.663772,4.765429,CA < US


In [41]:
movies['country_ptp_pair'].value_counts().head(25)

DE < US    673
SE < US    633
TR < US    234
DE < IN    220
DE < TR    167
GB < US    143
DE < BR    125
SE < GB    112
GB < TR    100
GB < IN     88
TR < BR     87
DE < GB     77
GB < BR     70
TR < IN     69
SE < DE     64
SE < CA     62
DE < NL     58
US < IN     54
DE < CA     51
CA < TR     50
TR < CA     49
SE < TR     48
SE < NL     44
CA < US     44
GB < DE     41
Name: country_ptp_pair, dtype: int64

In [ ]:
movies = movies.sort_values('polarization_score', ascending=False)